In [1]:
from selenium import webdriver
import time
from PIL import ImageGrab

In [2]:
import requests
from hashlib import md5

class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

In [3]:
Id = input('请输入账号：')
password = input('请输入密码：')
url = 'http://xkctk.hangzhou.gov.cn/'
driver = webdriver.Chrome()
driver.get(url)
# 将屏幕最大化，方便获取验证码
driver.maximize_window()
time.sleep(1)

请输入账号：aaaaa
请输入密码：aaaaa


# 截屏，获取验证码图片保存到本地  (此处该网页要在屏幕最前面)

In [14]:
bbox = (1459, 1020, 1575, 1055)
im = ImageGrab.grab(bbox)
im.save('a.jpg')

# 获取验证码

In [15]:
chaojiying = Chaojiying_Client('1477622930', 'tyj000324', '905123')	#用户中心>>软件ID 生成一个替换 96001
im = open('a.jpg', 'rb').read()  #本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
# print(chaojiying.PostPic(im, 1902))  #1902 验证码类型  官方网站>>价格体系 3.4+版 print 后要加()
yzm = chaojiying.PostPic(im, 1902)['pic_str']
print(yzm)

54mk


In [16]:
driver.find_element_by_name('loginCode').send_keys(Id)
driver.find_element_by_name('password').send_keys(password)
driver.find_element_by_name('validCode').send_keys(yzm)

# 点击登录

In [17]:
driver.find_element_by_id('userLoginButton').click()